In [2]:
from langchain_community.document_loaders import PyPDFDirectoryLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


In [3]:
loader=PyPDFDirectoryLoader("./us_census")

documents=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
final_docs=text_splitter.split_documents(documents)
final_docs[:5]

[Document(page_content='Occupation, Earnings, and Job \nCharacteristics\nJuly 2022P70-178Clayton Gumber and Briana SullivanCurrent Population Reports\nINTRODUCTION\nWork is a critical component of our lives and provides \na way to obtain material and nonmonetary benefits \nlike employer-provided health insurance. Scholars \nsuggest that our identities are also tied to the notion \nof “what we do” (Christiansen, 1999), and that who \nwe are is determined partly by our occupational iden -\ntity (Skorikov and Vondracek, 2011). However, work is time consuming—the American Time Use Survey \nshows that in 2017 workers spent an average 8.21 \nhours each day engaged in work and work-related activities (Bureau of Labor Statistics, 2018). Given the \noverarching centrality of work in daily life, research -\ners and policymakers have increasingly turned their attention to examining job quality.\nThough it is not easily defined, job quality can \nbroadly be described as the features of employ -', 

In [4]:
len(final_docs)

316

In [5]:
#Embeeeding 

hface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5", #senterce-transformers/all-MiniLM-16-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

/home/issac/project/envllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/issac/project/envllm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/issac/project/envllm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
import numpy as np
np.array(hface_embeddings.embed_query(final_docs[0].page_content))

array([ 5.96365519e-03,  2.69741248e-02, -1.88379344e-02, -2.22754497e-02,
        1.16274171e-02,  3.95190865e-02,  7.00702369e-02, -4.99902591e-02,
       -6.59820298e-03,  7.27157900e-03,  4.11885269e-02,  2.12356430e-02,
        2.69026728e-03, -2.84612924e-02, -3.23452018e-02,  3.30748572e-03,
        1.15507655e-02, -4.02457155e-02, -1.72409918e-02,  9.90439579e-03,
        2.10259985e-02, -1.60758998e-02, -2.70399563e-02,  2.20490922e-03,
        2.26793494e-02,  2.50464585e-02, -2.40352713e-02, -4.83506173e-02,
       -1.20919859e-02, -1.03770129e-01, -4.06865291e-02,  3.68829593e-02,
        7.00232759e-02,  5.09844162e-02,  5.21274991e-02,  2.69346051e-02,
       -9.03027784e-03,  5.35027832e-02, -1.20793330e-02,  8.18769634e-03,
        2.27775108e-02, -4.43279324e-03, -3.29854414e-02, -8.23435839e-04,
       -4.84039262e-02,  4.65029553e-02, -4.86759394e-02, -3.63179371e-02,
       -7.79526308e-02,  4.84887250e-02, -6.25632470e-03, -6.34023873e-03,
        3.74384671e-02,  

In [7]:
import configparser
import os
configParser = configparser.RawConfigParser()   
configFilePath = r'/home/issac/project/llm/.env'
configParser.read(configFilePath)

os.environ["LANCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=configParser.get('config','lan_api')
os.environ["LANGCHAIN_PROJECT"]='Grok project'
groq_api=configParser.get('config','grok_api')
hface_api=configParser.get('config','hface_api')

In [8]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_docs[:120],hface_embeddings)

In [9]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [10]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0xffff403d3bb0> search_kwargs={'k': 3}


In [11]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']=hface_api


In [18]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}

)


HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/mistralai/Mistral-7B-v0.1 (Request ID: BQ1ONMdCbQPm6oAwekDAO)

Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate

In [19]:
query="What is the health insurance coverage?"
hf.invoke(query)

HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/mistralai/Mistral-7B-v0.1 (Request ID: ijY8LF25OeHtAhLxi5fse)

Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate

In [20]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [21]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [22]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [23]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])

HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/mistralai/Mistral-7B-v0.1 (Request ID: AjPSlivjNljUsm5LmCDGx)

Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate